### CNN을 이용한 영상분석

#### 데이터 준비
MLP와 동일하게 mnist 데이터를 가져 옵니다  
CNN에서는 처음 이미지 형태를 그대로 쓰므로 MLP 와 같이 데이터를 모두 펴주는 작업을 하지 않습니다.  
대신 RGB 칼러로 표현하기 위한 한 개 차원을 추가 합니다. 여기선 흑백으로 인해 실제 값은 넣지 않습니다.

In [2]:
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((x_train.shape[0],28,28,1))/255
x_test = x_test.reshape((x_test.shape[0],28,28,1))/255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

/home/bwlee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


CNN을 만들어 봅니다. MLP 에서 얘기했던 것처럼 함수로 생성부분을 싸도록 하겠습니다.

In [3]:
def cnn_network():
    model = models.Sequential()

    # CNN 1층
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1))) # 32: channel, (3, 3): filter size
    model.add(layers.MaxPooling2D((2,2)))
    # CNN 2층
    model.add(layers.Conv2D(64, (3,3), activation='relu')) 
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.25))
    # CNN 2D 를 1D 로 변환
    model.add(layers.Flatten())
    # 1D 로 부터 MLP 생성    
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  
    
    return model

cnn = cnn_network()
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                102464    
__________

MLP와 동일하게 실행해보면 됩니다.  
이 경우 이전의 MLP에 비해 조금 성능이 향상되었습니다.

In [4]:
cnn.compile(optimizer="adagrad", loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 4s 68us/step - loss: 0.1452 - acc: 0.9548
Epoch 2/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.0624 - acc: 0.9807
Epoch 3/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.0508 - acc: 0.9845
Epoch 4/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.0437 - acc: 0.9863
Epoch 5/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.0394 - acc: 0.9879


In [5]:
loss, accuracy = cnn.evaluate(x_test, y_test)
print(loss, accuracy)

10000/10000 [==============================] - 0s 36us/step
0.03242523178511765 0.9896
